### konlpy를 사용한 형태소분리와 Word2Vec를 사용한 벡터변환

In [2]:
# # 한국어 전처리를 위한
!pip install konlpy
# !pip install Twitter
# !pip install tqdm
!pip install gensim
# !pip install --upgrade jupyter ipywidgets


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os # 운영 체제 관련 작업을 수행하기 위한 Python 표준 라이브러리 중 하나 -> 자바 환경 변수를 설정하기 위해 사용
import pandas as pd # 데이터 조작 및 분석을 위한 파이썬 라이브러리 -> 데이터를 데이터프레임으로 로드하고 조작하는 데 사용
import numpy as np # 배열 및 수치 연산을 수행하기 위한 라이브러리 -> 다차원 배열 및 수학적 함수를 다루는 데 사용
from konlpy.tag import Okt # Konlpy 라이브러리에서 제공하는 한국어 형태소 분석기 중 하나 ->  Okt를 초기화하는 데 사용
from gensim.models.word2vec import Word2Vec # Gensim 라이브러리에서 제공하는 Word2Vec 모델을 생성하기 위한 클래스 -> 단어를 고차원 벡터로 표현하여 단어 간 유사성 및 의미 관계를 파악

In [32]:
# 본인 컴퓨터와 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [33]:
# 데이터를 데이터프레임으로 로드
book_data = pd.read_csv('../cleaned_data.csv', encoding='utf-8')# 데이터 파일 경로
book_data

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,BOOK_INTRCN_CN,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 달빛천사 성우 이용신의 첫 번째 에세이,이용신 지은이,푸른숲,NaN,NaN,160000.0,httpsimagealadincokrproduct284158coverk6528351...,2004년 방영한 애니메이션 달빛천사에서 주인공 루나풀문 역을 맡으며 90년대생들에...,NaN,NaN,NaN,20211203.0,NaN,NaN,NaN
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 99년생 시인의 자의식 과잉 에세이,차도하 지은이,위즈덤하우스,NaN,NaN,158000.0,httpsimagealadincokrproduct2841466coverk202835...,그러니 나는 말하고 싶은 것을 말하겠다침착하게 사랑하기 차도하 시인 첫 에세이새롭고...,NaN,NaN,NaN,20211206.0,NaN,NaN,NaN
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 임오군란과 통킹 위기,굽시니스트 지은이,위즈덤하우스,NaN,NaN,148000.0,httpsimagealadincokrproduct2841447coverk402835...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다 러시아의 세력 확장을 ...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 신기하게 마음이 편해지는 응급 처방전,올리비아 레메스 지은이 김잔디 옮긴이,위즈덤하우스,NaN,NaN,140000.0,httpsimagealadincokrproduct2841430coverk892835...,누구에게나 기분 구급상자가 필요하다 하나씩 하나씩 차근차근 좋은 기분을 쌓고 건강한...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN
4,6352232,9791168120747,NaN,오늘도 리추얼 음악 나에게 선물하는 시간,정혜윤 지은이,위즈덤하우스,NaN,NaN,150000.0,httpsimagealadincokrproduct2841380coverk202835...,나다운 일상을 만드는 사람들의 이야기를 담은 오늘도 리추얼 시리즈가 위즈덤하우스에서...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,6354246,9791164670871,NaN,학교 서클대화가 필요해,손연일 심선화 장경아 지은이,북트리,NaN,NaN,160000.0,httpsimagealadincokrproduct2837599coverk262835...,평화로운 학교 공동체의 여정으로 가는 친절한 안내서 관계를 회복하고 배움이 일어나는...,NaN,학교서클대화가필요해,손연일심선화장경아지은이,20211125.0,Y,Y,NaN
1998,6354247,9788994229003,NaN,People Make City,이상환 지은이,디자인로커스,NaN,NaN,150000.0,httpsimagealadincokrproduct141865cover89942290...,따뜻한 성북동 만들기피블메이크시티시리즈는 대상지역의 문화 예술적 개발 방법론을 연구...,NaN,peoplemakecity,이상환지은이,20111121.0,Y,Y,NaN
1999,6354248,9788994027203,NaN,동방해경표,김려 지음,미디어버스,NaN,NaN,NaN,NaN,NaN,NaN,동방해경표,김려지음,NaN,Y,Y,NaN
2000,6354249,9791156759126,NaN,큰글자도서 그렇다면 정상입니다,하지현 지은이,푸른숲,NaN,NaN,360000.0,httpsimagealadincokrproduct2834689coverk292835...,심야 치유 식당 사랑하기에 결코 늦지 않았다 엄마의 빈틈이 아이를 키운다 등을 통해...,NaN,큰글자도서그렇다면정상입니다,하지현지은이,20211028.0,Y,Y,NaN


### 텍스트 전처리

In [34]:
# 형태소 분리를 위한 Konlpy 객체 초기화
twitter = Okt()

In [35]:
#텍스트를 전처리 하는 함수
#BOOK_INTRCN_CN: 전처리할 도서 줄거리
def preprocessingText(BOOK_INTRCN_CN):
    #전처리한 결과를 저장 할 리스트
    stems = []
    #text 를 단어로 나누고 품사를 붙여서 리턴
    #stem=True : 단어의 기본형으로 변환
    tagged_review = twitter.pos(BOOK_INTRCN_CN,stem=True)

    #tagged에서 하나를 꺼내서 word,pos에 대입
    #단어는 word, 품사는 pos에 대입
    for word,pos in tagged_review:
        #len(word) : 단어의 글자수가 2글자 이상이면
        #품사가 Noun(명사) 또는 Adjective(형용사) 일경우에만 word(단어) 를 stems에 추가
        if pos=="Noun" or pos=='Adjective':
            stems.append(word)

    #리스트인 stems를 문자열로 만들어서 ( "".join(stems) ) 리턴       
    return " ".join(stems)

In [36]:
preprocessingText(book_data['BOOK_INTRCN_CN'][0])

'방영 애니메이션 달빛천사 주인공 루나 풀문 역 보석 같다 추억 선물 성우 이용신 첫 에세이 수많다 작품 주연 쉬 대중 행복 전 성우 이용신 발자취 확인 수 있다'

In [37]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체 및 전처리
book_data['BOOK_INTRCN_CN_clear'] = book_data['BOOK_INTRCN_CN'].fillna('').apply(preprocessingText)

In [38]:
# Word2Vec 모델 훈련을 위한 데이터 준비
sentences = book_data['BOOK_INTRCN_CN'].tolist()

In [39]:
# 데이터프레임의 'BOOK_INTRCN_CN' 열에 NaN 값이 있을 때 빈 문자열로 대체
tokenized_data = book_data['BOOK_INTRCN_CN'].apply(lambda x: preprocessingText(str(x))).fillna('')

In [40]:
book_data

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,BOOK_INTRCN_CN,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO,BOOK_INTRCN_CN_clear
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 달빛천사 성우 이용신의 첫 번째 에세이,이용신 지은이,푸른숲,NaN,NaN,160000.0,httpsimagealadincokrproduct284158coverk6528351...,2004년 방영한 애니메이션 달빛천사에서 주인공 루나풀문 역을 맡으며 90년대생들에...,NaN,NaN,NaN,20211203.0,NaN,NaN,NaN,방영 애니메이션 달빛천사 주인공 루나 풀문 역 보석 같다 추억 선물 성우 이용신 첫...
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 99년생 시인의 자의식 과잉 에세이,차도하 지은이,위즈덤하우스,NaN,NaN,158000.0,httpsimagealadincokrproduct2841466coverk202835...,그러니 나는 말하고 싶은 것을 말하겠다침착하게 사랑하기 차도하 시인 첫 에세이새롭고...,NaN,NaN,NaN,20211206.0,NaN,NaN,NaN,그렇다 나 말 것 말 침착하다 사랑 차도 시인 첫 에세이 새 도발 작품 문단 대주 ...
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 임오군란과 통킹 위기,굽시니스트 지은이,위즈덤하우스,NaN,NaN,148000.0,httpsimagealadincokrproduct2841447coverk402835...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다 러시아의 세력 확장을 ...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,한중일 관계 결정 분기점 임오군란 막후 러시아 세력 확장 경계 청 친 결 미라 계책...
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 신기하게 마음이 편해지는 응급 처방전,올리비아 레메스 지은이 김잔디 옮긴이,위즈덤하우스,NaN,NaN,140000.0,httpsimagealadincokrproduct2841430coverk892835...,누구에게나 기분 구급상자가 필요하다 하나씩 하나씩 차근차근 좋은 기분을 쌓고 건강한...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,누구 기분 구급상자 필요하다 하나 하나 차근차근 좋다 기분 건강하다 마음 법 책
4,6352232,9791168120747,NaN,오늘도 리추얼 음악 나에게 선물하는 시간,정혜윤 지은이,위즈덤하우스,NaN,NaN,150000.0,httpsimagealadincokrproduct2841380coverk202835...,나다운 일상을 만드는 사람들의 이야기를 담은 오늘도 리추얼 시리즈가 위즈덤하우스에서...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,일상 사람 이야기 오늘 리추얼 시리즈 위즈 덤 하우스 간 오늘 리추얼 시리즈 음악 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,6354246,9791164670871,NaN,학교 서클대화가 필요해,손연일 심선화 장경아 지은이,북트리,NaN,NaN,160000.0,httpsimagealadincokrproduct2837599coverk262835...,평화로운 학교 공동체의 여정으로 가는 친절한 안내서 관계를 회복하고 배움이 일어나는...,NaN,학교서클대화가필요해,손연일심선화장경아지은이,20211125.0,Y,Y,NaN,평화롭다 학교 공동체 여정 친절하다 안내서 관계 회복하다 배움 평화롭다 학교 공동체...
1998,6354247,9788994229003,NaN,People Make City,이상환 지은이,디자인로커스,NaN,NaN,150000.0,httpsimagealadincokrproduct141865cover89942290...,따뜻한 성북동 만들기피블메이크시티시리즈는 대상지역의 문화 예술적 개발 방법론을 연구...,NaN,peoplemakecity,이상환지은이,20111121.0,Y,Y,NaN,따뜻하다 성북동 만들기 피블 메이크 시티 시리즈 대상 지역 문화 예술 개발 방법론 ...
1999,6354248,9788994027203,NaN,동방해경표,김려 지음,미디어버스,NaN,NaN,NaN,NaN,NaN,NaN,동방해경표,김려지음,NaN,Y,Y,NaN,
2000,6354249,9791156759126,NaN,큰글자도서 그렇다면 정상입니다,하지현 지은이,푸른숲,NaN,NaN,360000.0,httpsimagealadincokrproduct2834689coverk292835...,심야 치유 식당 사랑하기에 결코 늦지 않았다 엄마의 빈틈이 아이를 키운다 등을 통해...,NaN,큰글자도서그렇다면정상입니다,하지현지은이,20211028.0,Y,Y,NaN,심야 치유 식당 사랑 결코 엄마 빈틈 아이 등 통해 사람 문제 진단 그 적합하다 해...


In [41]:
#BOOK_INTRCN_CN_clear의 마지막에 " " 추가
book_data["BOOK_INTRCN_CN_clear"] = book_data['BOOK_INTRCN_CN_clear'].astype(str) + " "

In [42]:
#data.split(" ") : 문자열을 공백기준으로 잘라서 리스트로 변환
book_data["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" "))

0       [방영, 애니메이션, 달빛천사, 주인공, 루나, 풀문, 역, 보석, 같다, 추억, ...
1       [그렇다, 나, 말, 것, 말, 침착하다, 사랑, 차도, 시인, 첫, 에세이, 새,...
2       [한중일, 관계, 결정, 분기점, 임오군란, 막후, 러시아, 세력, 확장, 경계, ...
3       [누구, 기분, 구급상자, 필요하다, 하나, 하나, 차근차근, 좋다, 기분, 건강하...
4       [일상, 사람, 이야기, 오늘, 리추얼, 시리즈, 위즈, 덤, 하우스, 간, 오늘,...
                              ...                        
1997    [평화롭다, 학교, 공동체, 여정, 친절하다, 안내서, 관계, 회복하다, 배움, 평...
1998    [따뜻하다, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 대상, 지역, 문화,...
1999                                                 [, ]
2000    [심야, 치유, 식당, 사랑, 결코, 엄마, 빈틈, 아이, 등, 통해, 사람, 문제...
2001                                                 [, ]
Name: BOOK_INTRCN_CN_clear, Length: 2002, dtype: object

In [43]:
#df["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" ")) : BOOK_INTRCN_CN_clear 컬럼을 공백을 기준으로 단어별로 분리해서 리턴
book_data["tokenized_data"] = book_data["BOOK_INTRCN_CN_clear"].apply(lambda data : data.split(" "))

In [44]:
book_data

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,BOOK_INTRCN_CN,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO,BOOK_INTRCN_CN_clear,tokenized_data
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 달빛천사 성우 이용신의 첫 번째 에세이,이용신 지은이,푸른숲,NaN,NaN,160000.0,httpsimagealadincokrproduct284158coverk6528351...,2004년 방영한 애니메이션 달빛천사에서 주인공 루나풀문 역을 맡으며 90년대생들에...,NaN,NaN,NaN,20211203.0,NaN,NaN,NaN,방영 애니메이션 달빛천사 주인공 루나 풀문 역 보석 같다 추억 선물 성우 이용신 첫...,"[방영, 애니메이션, 달빛천사, 주인공, 루나, 풀문, 역, 보석, 같다, 추억, ..."
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 99년생 시인의 자의식 과잉 에세이,차도하 지은이,위즈덤하우스,NaN,NaN,158000.0,httpsimagealadincokrproduct2841466coverk202835...,그러니 나는 말하고 싶은 것을 말하겠다침착하게 사랑하기 차도하 시인 첫 에세이새롭고...,NaN,NaN,NaN,20211206.0,NaN,NaN,NaN,그렇다 나 말 것 말 침착하다 사랑 차도 시인 첫 에세이 새 도발 작품 문단 대주 ...,"[그렇다, 나, 말, 것, 말, 침착하다, 사랑, 차도, 시인, 첫, 에세이, 새,..."
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 임오군란과 통킹 위기,굽시니스트 지은이,위즈덤하우스,NaN,NaN,148000.0,httpsimagealadincokrproduct2841447coverk402835...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다 러시아의 세력 확장을 ...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,한중일 관계 결정 분기점 임오군란 막후 러시아 세력 확장 경계 청 친 결 미라 계책...,"[한중일, 관계, 결정, 분기점, 임오군란, 막후, 러시아, 세력, 확장, 경계, ..."
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 신기하게 마음이 편해지는 응급 처방전,올리비아 레메스 지은이 김잔디 옮긴이,위즈덤하우스,NaN,NaN,140000.0,httpsimagealadincokrproduct2841430coverk892835...,누구에게나 기분 구급상자가 필요하다 하나씩 하나씩 차근차근 좋은 기분을 쌓고 건강한...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,누구 기분 구급상자 필요하다 하나 하나 차근차근 좋다 기분 건강하다 마음 법 책,"[누구, 기분, 구급상자, 필요하다, 하나, 하나, 차근차근, 좋다, 기분, 건강하..."
4,6352232,9791168120747,NaN,오늘도 리추얼 음악 나에게 선물하는 시간,정혜윤 지은이,위즈덤하우스,NaN,NaN,150000.0,httpsimagealadincokrproduct2841380coverk202835...,나다운 일상을 만드는 사람들의 이야기를 담은 오늘도 리추얼 시리즈가 위즈덤하우스에서...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,일상 사람 이야기 오늘 리추얼 시리즈 위즈 덤 하우스 간 오늘 리추얼 시리즈 음악 ...,"[일상, 사람, 이야기, 오늘, 리추얼, 시리즈, 위즈, 덤, 하우스, 간, 오늘,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,6354246,9791164670871,NaN,학교 서클대화가 필요해,손연일 심선화 장경아 지은이,북트리,NaN,NaN,160000.0,httpsimagealadincokrproduct2837599coverk262835...,평화로운 학교 공동체의 여정으로 가는 친절한 안내서 관계를 회복하고 배움이 일어나는...,NaN,학교서클대화가필요해,손연일심선화장경아지은이,20211125.0,Y,Y,NaN,평화롭다 학교 공동체 여정 친절하다 안내서 관계 회복하다 배움 평화롭다 학교 공동체...,"[평화롭다, 학교, 공동체, 여정, 친절하다, 안내서, 관계, 회복하다, 배움, 평..."
1998,6354247,9788994229003,NaN,People Make City,이상환 지은이,디자인로커스,NaN,NaN,150000.0,httpsimagealadincokrproduct141865cover89942290...,따뜻한 성북동 만들기피블메이크시티시리즈는 대상지역의 문화 예술적 개발 방법론을 연구...,NaN,peoplemakecity,이상환지은이,20111121.0,Y,Y,NaN,따뜻하다 성북동 만들기 피블 메이크 시티 시리즈 대상 지역 문화 예술 개발 방법론 ...,"[따뜻하다, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 대상, 지역, 문화,..."
1999,6354248,9788994027203,NaN,동방해경표,김려 지음,미디어버스,NaN,NaN,NaN,NaN,NaN,NaN,동방해경표,김려지음,NaN,Y,Y,NaN,,"[, ]"
2000,6354249,9791156759126,NaN,큰글자도서 그렇다면 정상입니다,하지현 지은이,푸른숲,NaN,NaN,360000.0,httpsimagealadincokrproduct2834689coverk292835...,심야 치유 식당 사랑하기에 결코 늦지 않았다 엄마의 빈틈이 아이를 키운다 등을 통해...,NaN,큰글자도서그렇다면정상입니다,하지현지은이,20211028.0,Y,Y,NaN,심야 치유 식당 사랑 결코 엄마 빈틈 아이 등 통해 사람 문제 진단 그 적합하다 해...,"[심야, 치유, 식당, 사랑, 결코, 엄마, 빈틈, 아이, 등, 통해, 사람, 문제..."


In [45]:
book_data["tokenized_data"]

0       [방영, 애니메이션, 달빛천사, 주인공, 루나, 풀문, 역, 보석, 같다, 추억, ...
1       [그렇다, 나, 말, 것, 말, 침착하다, 사랑, 차도, 시인, 첫, 에세이, 새,...
2       [한중일, 관계, 결정, 분기점, 임오군란, 막후, 러시아, 세력, 확장, 경계, ...
3       [누구, 기분, 구급상자, 필요하다, 하나, 하나, 차근차근, 좋다, 기분, 건강하...
4       [일상, 사람, 이야기, 오늘, 리추얼, 시리즈, 위즈, 덤, 하우스, 간, 오늘,...
                              ...                        
1997    [평화롭다, 학교, 공동체, 여정, 친절하다, 안내서, 관계, 회복하다, 배움, 평...
1998    [따뜻하다, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 대상, 지역, 문화,...
1999                                                 [, ]
2000    [심야, 치유, 식당, 사랑, 결코, 엄마, 빈틈, 아이, 등, 통해, 사람, 문제...
2001                                                 [, ]
Name: tokenized_data, Length: 2002, dtype: object

### 유사도 측정
- 특정 줄거리와 다른 모든 줄거리 간의 유사도를 측정

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
import random

In [47]:
# 학습용 Word2Vec 모델 훈련(벡터화)
model = Word2Vec(book_data["tokenized_data"], 
                 vector_size=100, #WordVector의 열의 수
                 window=3, # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=2, # 사용할 단어의 최소 빈도(2회 이하 단어 무시)
                 sg=1) # 0: CBOW, 1: Skip-gram

In [48]:
# 모델을 저장할 경로와 파일 이름 지정
model.save("book_word2vec_model")

In [49]:
#Vector 형태로 변환한 단어 리스트 조회
model.wv.key_to_index

{'': 0,
 '있다': 1,
 '수': 2,
 '문제': 3,
 '학습': 4,
 '개념': 5,
 '구성': 6,
 '수록': 7,
 '책': 8,
 '수학': 9,
 '및': 10,
 '통해': 11,
 '주': 12,
 '것': 13,
 '독해': 14,
 '유형': 15,
 '연습': 16,
 '내용': 17,
 '등': 18,
 '다양하다': 19,
 '기출': 20,
 '주문': 21,
 '이해': 22,
 '이': 23,
 '교과': 24,
 '개': 25,
 '공부': 26,
 '별': 27,
 '단어': 28,
 '단계': 29,
 '과정': 30,
 '교재': 31,
 '속': 32,
 '방법': 33,
 '이야기': 34,
 '아이': 35,
 '시험': 36,
 '제공': 37,
 '위': 38,
 '시리즈': 39,
 '수능': 40,
 '어휘': 41,
 '완성': 42,
 '모의고사': 43,
 '실전': 44,
 '생활': 45,
 '핵심': 46,
 '연산': 47,
 '정리': 48,
 '재미있다': 49,
 '설명': 50,
 '어떻다': 51,
 '지문': 52,
 '작품': 53,
 '기본': 54,
 '영역': 55,
 '대한': 56,
 '학년': 57,
 '계산': 58,
 '적용': 59,
 '대비': 60,
 '권': 61,
 '만화': 62,
 '기초': 63,
 '그': 64,
 '필수': 65,
 '교과서': 66,
 '문장': 67,
 '저자': 68,
 '분석': 69,
 '체계': 70,
 '문학': 71,
 '집': 72,
 '가지': 73,
 '각': 74,
 '준비': 75,
 '위해': 76,
 '초등': 77,
 '법': 78,
 '풀이': 79,
 '교육': 80,
 '가능하다': 81,
 '같다': 82,
 '아니다': 83,
 '작가': 84,
 '해설': 85,
 '자료': 86,
 '글쓰기': 87,
 '확인': 88,
 '쓰기': 89,
 '중심': 90,
 '집중': 91,
 '

In [21]:
#믿음의 인덱스(빈도수 순위 0이 가장 빈도수 높은 단어)
model.wv.key_to_index["믿음"]

999

In [22]:
#믿음과 비슷한 벡터를 가진 단어 조회
model.wv.most_similar("믿음")

[('단위', 0.9932867288589478),
 ('숙어', 0.9931226968765259),
 ('자신', 0.9931122660636902),
 ('정화', 0.9930930733680725),
 ('대해', 0.9930409789085388),
 ('사랑', 0.9929673671722412),
 ('예문', 0.9929158091545105),
 ('시인', 0.9928885698318481),
 ('놀이', 0.9928585886955261),
 ('예술', 0.9928429126739502)]

In [23]:
#믿음의 WordVector 조회
model.wv["믿음"]

array([ 0.04277669,  0.06124747, -0.03916707,  0.02349655,  0.02676732,
       -0.08654908,  0.0306166 ,  0.0693801 ,  0.01783731, -0.0996289 ,
       -0.04342757, -0.05264195, -0.01539936,  0.01749153,  0.0618616 ,
       -0.0290568 ,  0.03104444, -0.0339325 ,  0.04011547, -0.02622598,
        0.07050281,  0.05003659, -0.00229809, -0.04612009, -0.02390318,
        0.03250837, -0.08073906, -0.07680172, -0.00520689,  0.07934827,
        0.05541853,  0.05652272, -0.00594931, -0.03381971,  0.03872221,
        0.08908868, -0.00785954, -0.04476715, -0.04734048, -0.05101632,
       -0.0386857 ,  0.01846601,  0.03589745, -0.05926029,  0.02639462,
       -0.08531541,  0.01444396,  0.01116299,  0.01974124,  0.08248001,
        0.02380429, -0.02424849, -0.00070164, -0.0155253 , -0.0431412 ,
        0.08056294,  0.08223881, -0.0682657 , -0.036117  , -0.01246204,
        0.05315188,  0.0374021 , -0.07084736, -0.01347209, -0.06021853,
        0.04612184,  0.0109287 ,  0.0176135 , -0.06202516,  0.07

In [24]:
#WordVector로 변환한 단어들 word2vec_words 대입
word2vec_words = model.wv.key_to_index.keys()
word2vec_words

dict_keys(['', '있다', '수', '문제', '학습', '개념', '구성', '수록', '책', '수학', '및', '통해', '주', '것', '독해', '유형', '연습', '내용', '등', '다양하다', '기출', '주문', '이해', '이', '교과', '개', '공부', '별', '단어', '단계', '과정', '교재', '속', '방법', '이야기', '아이', '시험', '제공', '위', '시리즈', '수능', '어휘', '완성', '모의고사', '실전', '생활', '핵심', '연산', '정리', '재미있다', '설명', '어떻다', '지문', '작품', '기본', '영역', '대한', '학년', '계산', '적용', '대비', '권', '만화', '기초', '그', '필수', '교과서', '문장', '저자', '분석', '체계', '문학', '집', '가지', '각', '준비', '위해', '초등', '법', '풀이', '교육', '가능하다', '같다', '아니다', '작가', '해설', '자료', '글쓰기', '확인', '쓰기', '중심', '집중', '사고력', '도움', '한눈', '하루', '반드시', '원리', '훈련', '최근', '삶', '능력', '강의', '중', '말', '마음', '서술', '실력', '해결', '난이도', '사람', '실제', '가능', '도서', '자신', '시간', '향상', '단', '영어', '부족', '문법', '대상', '흥미롭다', '소설', '요청', '언어', '세계', '재고', '식', '반영', '효과', '해외', '동화', '고객', '수입', '취소', '반품', '엄선', '제시', '불가하다', '원서', '대회', '경우', '우리', '소개', '사진', '최신', '학생', '전기', '출제', '학기', '없다', '독자', '서로', '북', '글', '동영상', '과목', '새롭다', '쪽', '간', '갈래', '스스로', '내신', '가장', '시

In [25]:
word2vec_words

dict_keys(['', '있다', '수', '문제', '학습', '개념', '구성', '수록', '책', '수학', '및', '통해', '주', '것', '독해', '유형', '연습', '내용', '등', '다양하다', '기출', '주문', '이해', '이', '교과', '개', '공부', '별', '단어', '단계', '과정', '교재', '속', '방법', '이야기', '아이', '시험', '제공', '위', '시리즈', '수능', '어휘', '완성', '모의고사', '실전', '생활', '핵심', '연산', '정리', '재미있다', '설명', '어떻다', '지문', '작품', '기본', '영역', '대한', '학년', '계산', '적용', '대비', '권', '만화', '기초', '그', '필수', '교과서', '문장', '저자', '분석', '체계', '문학', '집', '가지', '각', '준비', '위해', '초등', '법', '풀이', '교육', '가능하다', '같다', '아니다', '작가', '해설', '자료', '글쓰기', '확인', '쓰기', '중심', '집중', '사고력', '도움', '한눈', '하루', '반드시', '원리', '훈련', '최근', '삶', '능력', '강의', '중', '말', '마음', '서술', '실력', '해결', '난이도', '사람', '실제', '가능', '도서', '자신', '시간', '향상', '단', '영어', '부족', '문법', '대상', '흥미롭다', '소설', '요청', '언어', '세계', '재고', '식', '반영', '효과', '해외', '동화', '고객', '수입', '취소', '반품', '엄선', '제시', '불가하다', '원서', '대회', '경우', '우리', '소개', '사진', '최신', '학생', '전기', '출제', '학기', '없다', '독자', '서로', '북', '글', '동영상', '과목', '새롭다', '쪽', '간', '갈래', '스스로', '내신', '가장', '시

In [26]:
for index in range(2):
    # 도서 번호 조회
    NUM = book_data.loc[index,"SEQ_NO"]
    print("NUM=",NUM)
    # 도서 제목 조회
    TITLE = book_data.loc[index,"TITLE_NM"]
    print("TITLE=",TITLE)
    #줄거리 조회
    LINE = book_data.loc[index,"tokenized_data"]
    print("LINE=",LINE)
    doc2vec = None
    count = 0
    for word in LINE: #줄거리에서 단어 1개씩 리턴
        print("word=",word, end="\t")
        if word in word2vec_words: #단어가 WordVector에 포함된 단어 이면
            count += 1 #개수 1증가
            if doc2vec is None: #doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word] #단어의 WordVector 값 대입
            else: #첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word] #단어의 WordVector 값 추가

    if  doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count
    print()    
    print("="*100)
    print("NUM=",NUM,"TITLE=",TITLE,":doc2vec=",doc2vec)
    print("="*100)

NUM= 6352228
TITLE= 너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이
LINE= ['방영', '애니메이션', '달빛천사', '주인공', '루나', '풀문', '역', '보석', '같다', '추억', '선물', '성우', '이용신', '첫', '에세이', '수많다', '작품', '주연', '쉬', '대중', '행복', '전', '성우', '이용신', '발자취', '확인', '수', '있다', '']
word= 방영	word= 애니메이션	word= 달빛천사	word= 주인공	word= 루나	word= 풀문	word= 역	word= 보석	word= 같다	word= 추억	word= 선물	word= 성우	word= 이용신	word= 첫	word= 에세이	word= 수많다	word= 작품	word= 주연	word= 쉬	word= 대중	word= 행복	word= 전	word= 성우	word= 이용신	word= 발자취	word= 확인	word= 수	word= 있다	word= 	
NUM= 6352228 TITLE= 너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이 :doc2vec= [ 0.07679193  0.11078054 -0.08443831  0.02953975  0.07421557 -0.18665338
  0.05706055  0.13848396  0.02400837 -0.21357234 -0.1030007  -0.13030547
 -0.0239063   0.04714199  0.13812223 -0.08196609  0.06458025 -0.07498077
  0.08911119 -0.04525433  0.15815762  0.09291945 -0.01794691 -0.08742948
 -0.0313137   0.04726924 -0.1679002  -0.15017365 -0.03305832  0.16188407
  0.12920183  0.12730789  0.00107601 -0.08800846  0.06154756  

In [27]:
#행의 수를 row_num에 저장
row_num = len(book_data)
row_num

2002

### 벡터를 문자열로 저장

In [29]:
string_array = []  # 문자열을 저장할 빈 리스트

for index in range(row_num):
    # 도서 번호 조회
    NUM = book_data.loc[index, "SEQ_NO"]
    print("NUM=", NUM)
    # 도서 제목 조회
    TITLE = book_data.loc[index, "TITLE_NM"]
    print("TITLE=", TITLE)
    # 줄거리 조회
    LINE = book_data.loc[index, "tokenized_data"]
    print("LINE=", LINE)
    doc2vec = None
    count = 0

    for word in LINE:  # 줄거리에서 단어 1개씩 리턴
        print("word=", word, end="\t")
        if word in word2vec_words:  # 단어가 WordVector에 포함된 단어 이면
            count += 1  # 개수 1증가
            if doc2vec is None:  # doc2vect이 None(초기값 맨처음 실행 첫번째 단어) 이면
                doc2vec = model.wv[word]  # 단어의 WordVector 값 대입
            else:  # 첫번째 단어 아님
                doc2vec = doc2vec + model.wv[word]  # 단어의 WordVector 값 추가

    if doc2vec is not None:
        # 단어 벡터를 모두 더한 벡터의 값을 단어의 개수로 나눠줌
        doc2vec = doc2vec / count

    # 문자열로 변환해서 string_array 리스트에 추가
    string_array.append(doc2vec.tostring())

# string_array에 저장된 문자열을 데이터프레임에 추가
book_data["doc2vec_vector"] = string_array

NUM= 6352228
TITLE= 너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이
LINE= ['방영', '애니메이션', '달빛천사', '주인공', '루나', '풀문', '역', '보석', '같다', '추억', '선물', '성우', '이용신', '첫', '에세이', '수많다', '작품', '주연', '쉬', '대중', '행복', '전', '성우', '이용신', '발자취', '확인', '수', '있다', '']
word= 방영	word= 애니메이션	word= 달빛천사	word= 주인공	word= 루나	word= 풀문	word= 역	word= 보석	word= 같다	word= 추억	word= 선물	word= 성우	word= 이용신	word= 첫	word= 에세이	word= 수많다	word= 작품	word= 주연	word= 쉬	word= 대중	word= 행복	word= 전	word= 성우	word= 이용신	word= 발자취	word= 확인	word= 수	word= 있다	word= 	NUM= 6352229
TITLE= 일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이
LINE= ['그렇다', '나', '말', '것', '말', '침착하다', '사랑', '차도', '시인', '첫', '에세이', '새', '도발', '작품', '문단', '대주', '차도', '시인', '첫', '산문', '집', '간', '혼자', '일기', '거짓말', '수', '없다', '시인', '산문', '집', '일기', '거짓말', '사람', '공교', '롭', '도', '']
word= 그렇다	word= 나	word= 말	word= 것	word= 말	word= 침착하다	word= 사랑	word= 차도	word= 시인	word= 첫	word= 에세이	word= 새	word= 도발	word= 작품	word= 문단	word= 대주	word= 차도	word= 시인	word= 첫	word= 산문	word= 집	word= 간	word= 혼자	word= 일기	word= 

C:\Users\dsmdm\AppData\Local\Temp\ipykernel_9880\1285500749.py:30: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  string_array.append(doc2vec.tostring())


NUM= 6353707
TITLE= 인도 3
LINE= ['', '']
word= 	word= 	NUM= 6353708
TITLE= 레오는 정원사 22
LINE= ['', '']
word= 	word= 	NUM= 6353709
TITLE= 세계역사와 민담으로 만나는꼬불꼬불 문자 이야기
LINE= ['', '']
word= 	word= 	NUM= 6353710
TITLE= 동물이란 무엇인가 5
LINE= ['', '']
word= 	word= 	NUM= 6353711
TITLE= 뿌리깊은 나무 12
LINE= ['', '']
word= 	word= 	NUM= 6353712
TITLE= 파뇨파뇨 디지캐럿 스티커놀이 ㄱㄴㄷ
LINE= ['유아', '용', '한글', '학습', '스티커', '놀이', '북', '이', '교재', '애니메이션', '파', '뇨파뇨', '디지캐럿', '캐릭터', '스티커', '함', '한글', '구성', '스티커', '포함', '']
word= 유아	word= 용	word= 한글	word= 학습	word= 스티커	word= 놀이	word= 북	word= 이	word= 교재	word= 애니메이션	word= 파	word= 뇨파뇨	word= 디지캐럿	word= 캐릭터	word= 스티커	word= 함	word= 한글	word= 구성	word= 스티커	word= 포함	word= 	NUM= 6353713
TITLE= 누리와 미리의 우와 공룡이다
LINE= ['', '']
word= 	word= 	NUM= 6353714
TITLE= 서른살이 심리학에게 묻다
LINE= ['', '']
word= 	word= 	NUM= 6353715
TITLE= 알파걸들에게 주눅 든내 아들을 지켜라
LINE= ['', '']
word= 	word= 	NUM= 6353716
TITLE= 공부도둑  한 공부꾼의 자기 이야기
LINE= ['', '']
word= 	word= 	NUM= 6353717
TITLE= 엄마와 함께 읽는교과서 전래동화
LINE= ['', '']
wo

In [30]:
book_data

,SEQ_NO,ISBN_THIRTEEN_NO,VLM_NM,TITLE_NM,AUTHR_NM,PUBLISHER_NM,PBLICTE_DE,ADTION_SMBL_NM,PRC_VALUE,IMAGE_URL,...,KDC_NM,TITLE_SBST_NM,AUTHR_SBST_NM,TWO_PBLICTE_DE,INTNT_BOOKST_BOOK_EXST_AT,PORTAL_SITE_BOOK_EXST_AT,ISBN_NO,BOOK_INTRCN_CN_clear,tokenized_data,doc2vec_vector
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 달빛천사 성우 이용신의 첫 번째 에세이,이용신 지은이,푸른숲,NaN,NaN,160000.0,httpsimagealadincokrproduct284158coverk6528351...,...,NaN,NaN,NaN,20211203.0,NaN,NaN,NaN,방영 애니메이션 달빛천사 주인공 루나 풀문 역 보석 같다 추억 선물 성우 이용신 첫...,"[방영, 애니메이션, 달빛천사, 주인공, 루나, 풀문, 역, 보석, 같다, 추억, ...",b'\x16E\x9d=\xe8\xe0\xe2=\xfe\xed\xac\xbdX\xfd...
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 99년생 시인의 자의식 과잉 에세이,차도하 지은이,위즈덤하우스,NaN,NaN,158000.0,httpsimagealadincokrproduct2841466coverk202835...,...,NaN,NaN,NaN,20211206.0,NaN,NaN,NaN,그렇다 나 말 것 말 침착하다 사랑 차도 시인 첫 에세이 새 도발 작품 문단 대주 ...,"[그렇다, 나, 말, 것, 말, 침착하다, 사랑, 차도, 시인, 첫, 에세이, 새,...",b'\n\x8b\xa1=S\xdd\xeb=?E\xb4\xbdl\x9e\x03=\x0...
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 임오군란과 통킹 위기,굽시니스트 지은이,위즈덤하우스,NaN,NaN,148000.0,httpsimagealadincokrproduct2841447coverk402835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,한중일 관계 결정 분기점 임오군란 막후 러시아 세력 확장 경계 청 친 결 미라 계책...,"[한중일, 관계, 결정, 분기점, 임오군란, 막후, 러시아, 세력, 확장, 경계, ...",b'\x19\xb7\x87=\xeb!\xc1=\xec\xb5\x8f\xbd\xbb\...
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 신기하게 마음이 편해지는 응급 처방전,올리비아 레메스 지은이 김잔디 옮긴이,위즈덤하우스,NaN,NaN,140000.0,httpsimagealadincokrproduct2841430coverk892835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,누구 기분 구급상자 필요하다 하나 하나 차근차근 좋다 기분 건강하다 마음 법 책,"[누구, 기분, 구급상자, 필요하다, 하나, 하나, 차근차근, 좋다, 기분, 건강하...",b'j\xf5\x9f=\x95Z\xea=|\'\xaa\xbd\xa2\x92\xf2<...
4,6352232,9791168120747,NaN,오늘도 리추얼 음악 나에게 선물하는 시간,정혜윤 지은이,위즈덤하우스,NaN,NaN,150000.0,httpsimagealadincokrproduct2841380coverk202835...,...,NaN,NaN,NaN,20211201.0,NaN,NaN,NaN,일상 사람 이야기 오늘 리추얼 시리즈 위즈 덤 하우스 간 오늘 리추얼 시리즈 음악 ...,"[일상, 사람, 이야기, 오늘, 리추얼, 시리즈, 위즈, 덤, 하우스, 간, 오늘,...",b'\xe7S\xac=&S\xf3=r2\xc3\xbd\xce0\n=z\xdd\x95...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,6354246,9791164670871,NaN,학교 서클대화가 필요해,손연일 심선화 장경아 지은이,북트리,NaN,NaN,160000.0,httpsimagealadincokrproduct2837599coverk262835...,...,NaN,학교서클대화가필요해,손연일심선화장경아지은이,20211125.0,Y,Y,NaN,평화롭다 학교 공동체 여정 친절하다 안내서 관계 회복하다 배움 평화롭다 학교 공동체...,"[평화롭다, 학교, 공동체, 여정, 친절하다, 안내서, 관계, 회복하다, 배움, 평...",b'\xa6h\x95=f\x80\xd6=\xbf(\x9e\xbd?\xea\xe6<\...
1998,6354247,9788994229003,NaN,People Make City,이상환 지은이,디자인로커스,NaN,NaN,150000.0,httpsimagealadincokrproduct141865cover89942290...,...,NaN,peoplemakecity,이상환지은이,20111121.0,Y,Y,NaN,따뜻하다 성북동 만들기 피블 메이크 시티 시리즈 대상 지역 문화 예술 개발 방법론 ...,"[따뜻하다, 성북동, 만들기, 피블, 메이크, 시티, 시리즈, 대상, 지역, 문화,...",b'\xa1m\xb0=\xee\xcb\xf9=V_\xbb\xbd\xb4\\\x0e=...
1999,6354248,9788994027203,NaN,동방해경표,김려 지음,미디어버스,NaN,NaN,NaN,NaN,...,NaN,동방해경표,김려지음,NaN,Y,Y,NaN,,"[, ]","b""\xbf\x9a\xc7=\xfe\xab\r>2\xbb\xd3\xbd\x91\x0..."
2000,6354249,9791156759126,NaN,큰글자도서 그렇다면 정상입니다,하지현 지은이,푸른숲,NaN,NaN,360000.0,httpsimagealadincokrproduct2834689coverk292835...,...,NaN,큰글자도서그렇다면정상입니다,하지현지은이,20211028.0,Y,Y,NaN,심야 치유 식당 사랑 결코 엄마 빈틈 아이 등 통해 사람 문제 진단 그 적합하다 해...,"[심야, 치유, 식당, 사랑, 결코, 엄마, 빈틈, 아이, 등, 통해, 사람, 문제...","b""zt\xb2=5\xa7\xf0=\x9c3\xbc\xbd\xdfx\x10=\x94..."


In [31]:
book_data.to_csv('book_data_vector.csv', index=False, encoding='utf-8')